### Data Prep - July 28
#### Objective
1. Read in 4 csv files stored on box drive. For each...
1. Convert to dataframe
1. Remove rows missing XY daa
1. Convert remaining rows to spatial dataframe
1. Upload to AGOL

In [1]:
#Import packages
from datetime import datetime
import numpy as np
import pandas as pd
from arcgis import GIS
gis = GIS('pro')

In [2]:
#Check user
print (gis.users.me)

<User username:jpfay_dukeuniv>


In [3]:
#Read xlsx file listing filenames to process
df_files = pd.read_excel('Study and sample lists for ArcGIS Online.xlsx')

In [4]:
#Get folder location
box_folder = 'C:\\Users\\jpfay\\Box\\Files to Process\\USABLE Data\\NOT Processed\\'

In [52]:
for idx in [3]:
    row = df_files.loc[idx,:]
    filename = box_folder+row['File name']
    lat_fld = row['Latitude Field Name'].strip()
    lng_fld = row['Longitude Field Name']
    hg_fld = row['Hg Field Name']
    sample_type = row['Sample type']

    #Read in file
    df = pd.read_csv(filename,
                     usecols=[lat_fld,lng_fld,hg_fld],
                     dtype=np.double)
    total_rows = df.shape[0]
    df_files.loc[idx,'total records'] = total_rows
    print(f'{total_rows} total rows')

    #Remove rows missing values
    df_valid = df.dropna(axis='rows',how='any').reset_index()
    valid_rows = df_valid.shape[0]
    df_files.loc[idx,'valid records'] = valid_rows
    print(f'{valid_rows} valid rows')

    #Fix field names (remove spaces)
    #for fld in sdf.columns:
    #    if " " in fld: sdf.rename(columns={fld:fld.replace(' ','')},inplace=True)
    
    #Convert to spatial dataframe
    sdf = pd.DataFrame.spatial.from_xy(
        df = df_valid,
        x_column=lng_fld,
        y_column=lat_fld)

    #Convert to feature layer
    layer_name = row['File name'].split('.')[0]
    print(layer_name)
    feat_lyr = sdf.spatial.to_featurelayer(
        title = layer_name,
        tags = ['Peru'],
        folder = 'Peru_derived_layers'
    )
    #Share the layer
    feat_lyr.share('a81478c63d434e33829d2168c4dfdd93')

275 total rows
195 valid rows
FishAnalysisLagCategories_final


RuntimeError: The operation was attempted on an empty geometry.

In [72]:
sdf_Fix = sdf.rename(columns={hg_fld:'HgAvgFish_ppm',lat_fld:'Point_Y_latitude'})

In [73]:
feat_lyr = sdf_Fix.spatial.to_featurelayer(
    title = "fOO",
    tags = ['Peru'],
    folder = 'Peru_derived_layers'
)

In [64]:
feat_lyr.share('a81478c63d434e33829d2168c4dfdd93')

{'results': [{'itemId': '2839bc5d6f9a42c1a7f21afd2f0a5642',
   'success': True,
   'notSharedWith': []}]}

In [65]:
df_files.to_csv('Processed.csv',index=False)

In [66]:
df_files

,Study,File name,Sample type,Latitude Field Name,Longitude Field Name,Hg Field Name,total records,valid records
0,"Hunt Study, IOH Study, RioMDD Study",cleaned_HGMAPS.csv,Humans,LAT,LON,HAIR_HG,3259.0,3259.0
1,River Study,suspendedparticles_final.csv,Suspended particles in water,Point_Y,Point_X,Hg_ss_avg,77.0,66.0
2,River Study,sediment_analysis_final.csv,Sediment,Latitude,Longitude,hgt_avg,74.0,59.0
3,El Nino Study,FishAnalysisLagCategories_final.csv,Fish,Point_Y_ latitude,Point_X_longitude,HgAvgFish (ppm),275.0,195.0


In [67]:
import os
os.getcwd()

'C:\\Workspace\\Gits'

In [77]:
fix_fields = [[fld,fld.replace(" ","")] for fld in sdf.columns if " " in fld]

In [81]:
for fld in sdf.columns:
    if " " in fld: 
        fix_fld = fld.replace(' ','')
        print(fld)
        sdf.rename(columns={fld:fix_fld},inplace=True)

HgAvgFish (ppm)
Point_Y_ latitude
